<a href="https://colab.research.google.com/github/medaminechafii/spaceship-titanic/blob/main/mynewproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
data = pd.read_csv("train.csv")
X = data.iloc[:,:-1]
y = data.iloc[:,-1]